<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_19_Web_Search_to_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, and then search for relevant results and respond based on user prompt.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

# Create specification for OpenAI GPT-4o Mini
async def create_completion_specification(content_limit: Optional[int] = None):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="OpenAI GPT-4o",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=enums.OpenAIModels.GPT4O_MINI_128K,
        ),
        strategy=input_types.ConversationStrategyInput(
            embedCitations=True
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION,
            contentLimit=content_limit if content_limit is not None else 5
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(search_text: str, read_limit: int):
    if graphlit.client is None:
        return;

    # NOTE: Uncomment one of these to select the search service
    service_type = enums.SearchServiceTypes.TAVILY
    #service_type = enums.SearchServiceTypes.EXA

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            type=service_type,
            text=search_text,
            readLimit=read_limit
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def create_conversation(specification_id: str, feed_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        ),
        filter=input_types.ContentCriteriaInput(
            feeds=[
                input_types.EntityReferenceInput(id=feed_id)
            ]
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_conversation(conversation_id: str):
    if graphlit.client is None:
        return;

    if conversation_id is not None:
        _ = await graphlit.client.delete_conversation(conversation_id)

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        message = response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
        citations = response.prompt_conversation.message.citations if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None

        return message, citations
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

async def publish_contents(feed_id: str, specification_id: str, prompt: str, content_limit: Optional[int] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.publish_contents(
            name="Summary",
            connector=input_types.ContentPublishingConnectorInput(
               type=enums.ContentPublishingServiceTypes.TEXT,
               format=enums.ContentPublishingFormats.MARKDOWN
            ),
            summary_specification=input_types.EntityReferenceInput(
                id=specification_id
            ),
            publish_prompt = prompt,
            publish_specification=input_types.EntityReferenceInput(
                id=specification_id
            ),
            filter=input_types.ContentFilter(
                limit=content_limit if content_limit is not None else 5,
                search=prompt,
                searchType=enums.SearchTypes.VECTOR,
                feeds=[
                    input_types.EntityReferenceFilter(id=feed_id)
                ]
            )
        )

        return response.publish_contents.markdown if response.publish_contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [ ]:
from IPython.display import display, Markdown, Image
import time

# Remove any existing specifications, conversations and workflows; only needed for notebook example
await delete_all_specifications()
await delete_all_conversations()
await delete_all_feeds()

print('Deleted all specifications, conversations and feeds.')

read_limit = 25 # how many search results to ingest from feed

prompt = "What are the current top investment opportunities in private aviation?"

feed_id = await create_feed(prompt, read_limit)

if feed_id is not None:
    print(f'Created feed [{feed_id}].')

    # Wait for feed to complete, since ingestion happens asychronously
    done = False
    time.sleep(5)
    while not done:
        done = await is_feed_done(feed_id)

        if not done:
            print('.', end='')
            time.sleep(2)

    print()
    print(f'Completed feed [{feed_id}].')

Deleted all specifications, conversations and feeds.
Created feed [6bf0102b-4338-4e44-a8b0-523657adb99d].
..........................................................................................................................................................................................
Completed feed [6bf0102b-4338-4e44-a8b0-523657adb99d].


In [ ]:
    content_limit = 10 # NOTE: Number of ingested web pages to inject into LLM context window

    specification_id = await create_completion_specification(content_limit)

    if specification_id is not None:
        print(f'Created specification [{specification_id}].')

        conversation_id = await create_conversation(specification_id, feed_id)

        if conversation_id is not None:
            print(f'Created conversation [{conversation_id}].')

            message, citations = await prompt_conversation(conversation_id, prompt)

            if message is not None:
                display(Markdown('### Conversation:'))
                display(Markdown(f'**User:**\n{prompt}'))
                display(Markdown(f'**Assistant:**\n{message}'))
                print()

                if citations is not None:
                    for citation in citations:
                        if citation is not None and citation.content is not None:
                            display(Markdown(f'**Citation [{citation.index}]:** {citation.content.name}'))
                            display(Markdown(citation.text))
                            print()

            await delete_conversation(conversation_id)

Created specification [47a4d85a-bdcf-4bea-944c-0e289cd03100].
Created conversation [23e0f4b9-72c0-4539-a391-8f4a22a58d39].


### Conversation:

**User:**
What are the current top investment opportunities in private aviation?

**Assistant:**
The private aviation sector is witnessing a significant shift towards aircraft leasing, with nearly 50% of the global commercial aircraft fleet now leased rather than owned outright. This trend is driven by airlines seeking flexibility and reduced financial risk, making operating leases an attractive option for managing fleet requirements. [1][2][3][4][5]

Private equity is increasingly drawn to the aviation industry due to its strong growth potential and the attractive supply-demand dynamics. With air traffic projected to double over the next two decades, private equity firms are investing heavily in aircraft leasing companies and related infrastructure, positioning themselves to capitalize on the industry's expansion. [6][7][8][9][10]

Investments in aircraft financing are gaining traction as the post-pandemic recovery in air travel continues. The demand for in-service aircraft is expected to rise, providing opportunities for private credit strategies that focus on leasing aircraft to airlines, particularly in emerging markets where air travel demand is surging. [2][4][5][8][7]

The ongoing supply chain challenges faced by aircraft manufacturers have created a backlog in new aircraft deliveries, further enhancing the appeal of leasing options. Investors are looking at midlife aircraft, which offer a balance of cost and cash flow potential, as a viable investment strategy in the current market. [6][1][5][9][3]

The rise of fractional ownership and membership programs in private aviation is attracting new investors. These models not only democratize access to private flying but also create stable revenue streams for operators, making them an appealing investment opportunity in the evolving landscape of private aviation. [4][5][2][7][8]



**Citation [1]:** https://www.worldfinance.com/infrastructure-investment/private-equity-takes-flight-in-the-aviation-sector

...

Overall investment outlook for global aviation finance
Our exclusive survey of senior executives who have financed the aviation industry in the past three years reveals that the overall investment outlook for the sector is bright, with one region in particular set for larger growth than its global equivalents Insight 15 January 2020 | 3 min read Justin Benson | Christian Hansen
Subscribe
Stay current on your favorite topics
61%
of respondents expect investment in the global aviation sector to increase in 2020
The outlook for investment in the aviation sector remains encouraging, despite a backdrop of mounting fears about a global economic slowdown amid various geopolitical tensions. However, while the big picture is one of cautious optimism, the mood varies markedly around the world, with some markets much less upbeat.
As our survey findings reveal, more than half of the respondents in this research (61%) expect investment in the sector to increase during 2020; only 26% anticipate a decline in investment.
But these headline figures mask significant regional variations. In the APAC region, more than three-quarters of respondents expect to see increased investment, while in EMEA and North America, less than half of respondents agree. Indeed, in both EMEA and North America, almost as many respondents foresee investment falling back over the next 12 months.
Half of the private equity firms in this research expect to invest US$1 billion to US$5 billion in the sector over the next year, with airlines and operating lessors also committed to significant levels of funding.
While overall investment in aviation and aircraft financing may be likely to increase, the flow to different regions could be disproportionate. In Asia, for instance, a large, growing middle class population is supporting demand for air travel, while emerging low-cost carriers offer lower fares. In contrast, within Europe, a combination of economic stagnancy and Brexit is having a sizeable impact on the industry.
Expectations of activity in the aircraft financing market are also mixed. Overall, 61% of respondents anticipate increased activity during 2020, while only 22% predict a decline. However, APAC-based respondents are predominantly optimistic about the outlook, with 83% predicting an increase in aircraft finance, while only 55% of their counterparts in North America say the same. Meanwhile, sentiment in EMEA is depressed, with the number of respondents who expect reduced aircraft financing activity this year (44%) higher than those who look forward to an increase (37%).
Do you think overall investment in the global aviation sector will increase, decrease or remain the same during 2020? (PDF)
Inevitably, the contrast between APAC and Western markets—EMEA in particular—reflects a broader economic context. The International Monetary Fund's (IMF) latest World Economic Outlook report suggests the US economy is on course to grow by just 2.1% in 2020, due in part to the US-China trade dispute. In Europe, where the Brexit negotiations continue to cause uncertainty, the IMF's prediction is for only 1.4% growth in 2020. Asia's economies, by contrast, are expected to grow considerably more quickly, led by China, albeit at a slower pace than in recent years.
Local factors are also at play. For example, Europe's travel sector endured a torrid 12 months with a string of airline and travel industry failures, including the UK's Thomas Cook and FlyBMI, Germania, Iceland's Wow Air and Slovenia's Adria Airways. In the Middle East, concerns remain that the rapid expansion of the region's airlines may have resulted in overcapacity.
APAC, on the other hand, is where most aircraft are expected to be delivered over the next ten to 20 years.
Do you expect aircraft financing in the global aviation sector to increase, decrease or remain the same during 2020? (PDF)
What is your annual global aviation capital investment? (PDF)
Overall, there likely will be considerable capital flows into the aviation sector globally during 2020, with investment coming from a mixture of sources. Half of the private equity firms in this research expect to invest US$1 billion to US$5 billion in the sector over the next year, with airlines and operating lessors also committed to significant levels of funding.
"The demand for airline services has created long-term potential for investors," remarks the Director of Finance at one EMEA-based airline, who predicts continuing interest from private equity and others.
In the financing market, banks remain committed to providing significant sums to the aviation industry. Indeed, the aircraft financing environment remains remarkably robust, with investors attracted to a market that has seen passenger numbers grow every year since 2010. There are tremendous amounts of liquidity, and competition for assets and talent is fierce.
What is your annual global aviation finance investment? (PDF)
Click here to read the full magazine
Navigating turbulence
This publication is provided for your convenience and does not constitute legal advice. This publication is protected by copyright.
© 2020 White & Case LLP
Contacts Justin Benson Partner | London Services Debt Finance , Asset Finance , Aviation , Maritime & Offshore , Financial Institutions Christian Hansen Partner | Mexico City Services Mergers & Acquisitions , Debt Finance , Asset Finance , Aviation , Maritime & Offshore
Service areas Aviation Asset Finance Subscribe Overall%20investment%20outlook%20for%20global%20aviation%20finance
https://www.whitecase.com/insight-our-thinking/overall-investment-outlook-global-aviation-finance
https://news.whitecase.com/5/38/forms/subscribe.asp mailto:?subject=Overall%20investment%20outlook%20for%20global%20aviation%20finance&body=https://www.whitecase.com/insight-our-thinking/overall-investment-outlook-global-aviation-finance https://www.linkedin.com/shareArticle?mini=true&url=https://www.whitecase.com/insight-our-thinking/overall-investment-outlook-global-aviation-finance&title=Overall%20investment%20outlook%20for%20global%20aviation%20finance&source=www.whitecase.com https://twitter.com/intent/tweet?text=Overall%20investment%20outlook%20for%20global%20aviation%20finance&url=https://www.whitecase.com/insight-our-thinking/overall-investment-outlook-global-aviation-finance&via=WhiteCase https://www.facebook.com/share.php?u=https://www.whitecase.com/insight-our-thinking/overall-investment-outlook-global-aviation-finance&t=Overall%20investment%20outlook%20for%20global%20aviation%20finance https://news.whitecase.com/5/38/forms/subscribe.asp Top

**Citation [9]:** https://thenationonlineng.net/driving-investment-opportunities-in-aviation/

Driving investment opportunities in aviation
Yadudu
Investment opportunities are becoming boundless in aviation industry. Regrettably, private sector players in Nigeria  are not taking advantage of existing windows. To reverse this trend, regulators and the airport authority  are meeting in Abuja next month to examine ways of unlocking huge investment windows for the sector, KELVIN OSA OKUNBOR reports.
Global aviation is becoming a huge industry with  massive investment opportunities spanning airport business, airlines,  ground handling, shuttle and hailing services, airline catering, tourism, leasing, financing and other  activities that  boost economic development.
Studies reveal that public financing options are becoming inadequate to drive these chain of activities, thereby paving the way for massive private sector investment and other intervention models.
To bridge this gap, a campaign is mounting among aeronautical agencies on the strategies to be adopted to attract private sector investors.
The push for private sector investment continues to gain global traction as regulatory bodies, including Airports Council International (ACI) and International Air Transport Association (IATA), have endorsed the private sector investment model for countries, including Nigeria.
According to the International Air Transport Association (IATA), the global aviation industry supports over $2.7 trillion of the world’s gross domestic product.
On the African continent, operations of airlines and airports generate over $10.3 billion yearly to the gross domestic product.
The global airline’s regulator said air transport supports over 6.2 million jobs and $ 55.8 billion in Africa.
But, Nigeria is yet to fully plug into this window given the size of its aviation.
To reverse this trend, the government, through the Ministry of Aviation and Federal Airport Authority of Nigeria (FAAN), will engage players and stakeholders next month in Abuja in discussions on how to pull investment into the  aviation industry at  the maiden edition of National Aviation Conference.
In an interview with The Nation last week in Lagos,  Managing Director, FAAN, Captain Rabiu  Yadudu, said its proposed conference could not have come at a better time than when aeronautical authorities were thinking out of the box in  deepening the  space for greater private sector inclusion and investments in aviation and allied sectors.
He said the conference would provide a window for Nigeria to interface global air transport expertise in navigating around issues on the implementation of single African air transport market in terms of its prospects and challenges.

**Citation [10]:** https://www.worldfinance.com/infrastructure-investment/private-equity-takes-flight-in-the-aviation-sector

Growth within growth
Investors are investing in a growth industry within a growth industry. Driven by air travel demand, the latest Boeing and Airbus forecasts show that the commercial aircraft fleet is expected to more than double over the next two decades from its current level of roughly 20,000 commercial aircraft. Retirements of aircraft at the end of their economic lives (approximately 25 to 30 years) and their replacement by more fuel-efficient types further drives the demand for new aircraft production.
In a rising fuel price environment, replacement demand for fuel inefficient aircraft is strong. Whereas fuel cost as a share of total operating expenses was in the mid teens in the years 2003/2004, it has now risen to a stable 30 percent share, further driving replacement demand for older aircraft. Even in an environment with lower fuel prices, this trend is expected to continue. The introduction of more fuel-efficient new technology aircraft will de-risk the negative effects on airlines’ operations in the event of a fuel price spike.
Another attraction from an investor’s perspective in this space is that the two largest suppliers to the industry are relatively constrained in terms of their production capacity. Changes to production, whether it is an increase or decrease, is difficult given the long lead times involved and their supply chain capacity constraints. Extensive production backlogs on the most popular models allow Boeing and Airbus to manage their production levels as regional or individual customer demand shocks can be evened out by reallocation of order slots to other customers.
Private equity investors haven taken into account what are significant hurdles to entry into the aircraft leasing space, given that the businesses are very capital intensive with acceptable returns only provided on a levered basis. Private equity investors are familiar with different leverage options making them natural investors in the aircraft leasing space. They often apply a diversified debt funding mix to the lessor with funding ranging from the standard bank financings to export credit supported loans or even the US capital markets.
Investors are also drawn to this industry by the fact that investments in operating lessors are in long-lived assets. The value of these assets is relatively predictable. It is also supported by the solid fundamentals given the attractive supply/demand dynamic around aircraft capacity. Relatively predictable new aircraft production levels and a steady outlook in terms of growth and aircraft replacement demand drives such a dynamic. There is volatility in aircraft values, driven by supply/demand and competitive factors. However that volatility is mitigated when looking at aircraft values on a lease encumbered basis, where the lease cash flows and aircraft residual values mitigate a pure asset value view.
Similar to investments in real estate, large chunks of equity from investors can be deployed efficiently and quickly, with the most prevalent narrow body aircraft costing upwards to $50m and widebody aircraft prices starting from the low $100m per aircraft. However security lies in the fact that the assets can be moved from one operator to the next, should that be required at the end of a lease be it scheduled or unscheduled. The worldwide mobility of aircraft to different airlines around the world provides tremendous risk mitigation to the investor.
Aircraft leasing businesses are capital-intensive operations and the forward orders with Airbus and Boeing carry significant cash drag through the associated pre-delivery payments. In addition, there is placement risk for the aircraft, interest rate risk and financing risk that need to be managed. For that reason, many lessors tend to focus on sale-lease back transactions or trades from other lessors where all of these risks can be mitigated since the aircraft are already on lease to an operator and financing requirements are near term with very limited interest rate risk exposure.
Track record
Even though the investor focus draws comfort from the underlying asset values, private equity investors tend to invest in businesses and management teams rather than pools of assets themselves. This approach is largely based on the realisation that the aircraft leasing business is a relationship business, where airline and supplier relationships allow for the best realisation of value in transactions.
Private equity firms have now branched out into other aviation leasing areas, such as helicopters, as shown by the $375m of equity capital that funds associated with MSD Capital, Soros and Cartesian Capital Group have committed to helicopter lessor Waypoint Leasing. The attraction for private equity investors to the aircraft leasing business will likely remain strong given the solid fundamentals around asset values, the favourable and predictable supply and demand characteristics of aircraft assets and the long-term steady returns that lessors have provided throughout various industry cycles.
Most importantly, a positive track record by private equity investments in the aircraft leasing space has been established with several successful investment liquidations, whether private or into the public space. Private equity certainly has an important role to play to capitalise the aircraft lessors that are playing the economic cycles and investments in diverse aircraft assets in various ways. We should certainly be looking for more activity by private equity in the space. Related: From tradition to transformation in the luxury market Tackling the world’s hidden-debt problem Completing the banking puzzle Can extreme weather events threaten the rise of solar? Too much power: the problem of private equity Navigating cryptocurrency regulations Unlocking growth in the experience economy Can immigration fix Canada’s economy? Unprecedented rain in Dubai raises questions The next phase of digital banking

In [ ]:
        published_markdown = await publish_contents(feed_id, specification_id, prompt, content_limit)

        if published_markdown is not None:
            display(Markdown(published_markdown))

### Current Top Investment Opportunities in Private Aviation

1. **Aircraft Leasing**: The trend towards leasing rather than owning aircraft is gaining momentum. Nearly 50% of the world's commercial aircraft fleet is now under operating leases, which provide airlines with flexibility and reduced financial risk. Private equity firms are increasingly investing in aircraft leasing companies due to the stable cash flow and lower correlation with market volatility.

2. **Private Equity Investments**: Private equity is playing a significant role in the aviation sector, particularly in mergers and acquisitions of established businesses like fixed base operators (FBOs) and maintenance, repair, and overhaul (MRO) services. The general aviation industry is seeing substantial investments from private equity firms, driven by stable cash flows and growth potential.

3. **Aircraft Financing**: Investing in aircraft financing offers steady cash flow and returns that are less correlated with broader market indices. The post-pandemic recovery in air travel, driven by a growing global middle class, presents a unique opportunity for investors in this space.

4. **Fractional Ownership and Jet Cards**: Corporate flight departments are increasingly exploring fractional ownership and jet card programs as alternatives to outright ownership. This shift reflects a growing demand for flexible private aviation solutions, creating investment opportunities in these service models.

5. **Emerging Markets**: The Asia-Pacific region is showing significant optimism for investment growth in aviation, driven by rising incomes and increased travel demand. This region presents long-term investment potential, particularly in aircraft leasing and financing.

6. **Sustainable Aviation**: There is a growing interest in sustainable aviation practices, including the development of sustainable aviation fuel (SAF) and advanced air mobility (AAM). While these areas are more suited for venture capital, they represent future investment opportunities as the industry evolves.

7. **Technology and Innovation**: The aviation sector is increasingly recognizing the importance of technology in enhancing operational efficiency and passenger experience. Investments in technology solutions that improve safety and efficiency are becoming critical.

8. **Market Diversification**: Companies like Pen Aviation are offering diverse investment opportunities that span both aviation and non-aviation sectors. This diversification can help mitigate risks and enhance returns for investors.

9. **Regulatory Support**: The push for private sector investment in aviation is gaining momentum, with regulatory bodies supporting initiatives that foster economic growth. This regulatory environment can create favorable conditions for investment.

10. **Networking and Collaboration**: Engaging with industry stakeholders and participating in conferences can provide valuable insights and opportunities for investment. Companies like Pen Aviation emphasize the importance of networking to connect investors with potential projects.

These investment opportunities reflect the evolving landscape of private aviation, driven by changing consumer preferences, economic growth, and technological advancements.